In [1]:
print("pk")

pk


In [2]:
%pwd

'c:\\Users\\nandh\\Chatbot\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\nandh\\Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 39994


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\nandh\AppData\Local\Temp\ipykernel_43412\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\nandh\anaconda3\envs\medicalbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
from dotenv import load_dotenv
import os

# Explicitly specify the path to the .env file
load_dotenv("C:/Users/nandh/Chatbot/.env")


True

In [15]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [20]:

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key= PINECONE_API_KEY)
index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [21]:
import os 
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY


In [22]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [23]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [24]:
docsearch


In [25]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [26]:
retrieved_docs = retriever.invoke("What is Acne?")

In [27]:
retrieved_docs

[Document(id='ea902416-e7dc-45a1-a215-401e9dcad4b1', metadata={'page': 55.0, 'source': 'Data\\The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf'}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='17de325b-3c4e-45d6-a0ad-800dc8e96ab6', metadata={'page': 55.0, 'source': 'Data\\The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf'}, page_content='Sebaceous follicles— A structure found within the\nskin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nSebum— An oily skin moisturizer produced by\nsebaceous glands.\nTretinoin— A drug that works by increasing the\nturnover (death and replacement) of skin cells.\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous glands\nbecome inflamed.(Photograph by Biophoto Associates, Photo'),
 Document(id='eff35843-09a1-4627-a1ea-f92404211513', meta

In [28]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')



In [29]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv("C:/Users/nandh/Chatbot/.env")

# Fetch the API key from the environment
api_key = os.getenv("OPENAI_API_KEY")



In [30]:
import os 
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [31]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [34]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])


Stats, short for statistics, is a branch of mathematics that deals with collecting, organizing, analyzing, and interpreting numerical data. It involves using various methods and techniques to summarize and make sense of large sets of data. In the context of psychological assessment, statistics are used to establish norms and standards for tests and to evaluate an individual's performance in relation to these norms.


In [35]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])



Acromegaly and gigantism are disorders caused by an abnormal release of a chemical from the pituitary gland, which leads to increased growth in bone and soft tissue, as well as other disturbances in the body. These disorders can be confirmed through growth hormone tests, which can determine if the pituitary gland is underactive or overproducing. If left untreated, the disease does not worsen.


In [37]:
response = rag_chain.invoke({"input": "For what questions you say don't know ?"})
print(response["answer"])



I don't know for questions about the severity and effects of a person's alcohol or substance use, or for questions about evaluating the seriousness of a patient's intentions for suicide.


In [39]:
response = rag_chain.invoke({"input": "What is Fingertip injuries ?"})
print(response["answer"])



Fingertip injuries refer to any type of trauma or damage that occurs to the fingertips, such as cuts, bruises, bone fractures, or amputations. These injuries can be caused by various factors, including sports-related injuries or accidents. Treatment for fingertip injuries may involve cleaning and dressing the wound, or in more severe cases, amputation may be necessary. 
